# SIS with 4x4 matrices
Conjecture: 4x4 matrices are enough to capture SIS models exactly.

Compare with exact result on a small tree.

In [13]:
using MatrixProductBP, MatrixProductBP.Models, IndexedGraphs, Test, Random
import MatrixProductBP: nstates
import MatrixProductBP.Models: prob_xy, prob_yy, prob_y

In [24]:
T = 5

A = [0 1 0 0;
     1 0 1 1;
     0 1 0 0;
     0 1 0 0]
g = IndexedGraph(A)
N = size(A, 1)

λ = 0.5
ρ = 0.4
γ = 0.5;

In [25]:
sis = SIS(g, λ, ρ, T; γ)
bp = mpbp(sis)
rng = MersenneTwister(111)
X, _ = onesample(bp; rng)
draw_node_observations!(bp.ϕ, X, N, last_time=true; rng);

In [29]:
svd_trunc = TruncBond(4)
redirect_stdio(stdout="svd_log.txt") do
    iterate!(bp, maxiter=10; svd_trunc, showprogress=false, svd_verbose=true)
end;

In [30]:
b_bp = beliefs(bp)
p_bp = [[bbb[2] for bbb in bb] for bb in b_bp]

p_exact, Z_exact = exact_prob(bp)
b_exact = exact_marginals(bp; p_exact)
p_ex = [[bbb[2] for bbb in bb] for bb in b_exact]

f_bethe = bethe_free_energy(bp)
Z_bp = exp(-f_bethe)

f(x,i) = x-1

r_bp = autocorrelations(f, bp)
r_exact = exact_autocorrelations(f, bp; p_exact)

c_bp = autocovariances(f, bp)
c_exact = exact_autocovariances(f, bp; r = r_exact);

┌ Warning: T*N=20. This will take some time!
└ @ MatrixProductBP /home/stecrotti/Desktop/PhD/MatrixProductBP/src/exact.jl:8
Computing joint probability 100%|████████████████████████| Time: 0:00:55
Computing exact marginals 100%|██████████████████████████| Time: 0:02:01
Computing exact marginals 100%|██████████████████████████| Time: 0:02:03
┌ Warning: T*N=20. This will take some time!
└ @ MatrixProductBP /home/stecrotti/Desktop/PhD/MatrixProductBP/src/exact.jl:8
Computing joint probability 100%|████████████████████████| Time: 0:00:54
Computing exact marginals 100%|██████████████████████████| Time: 0:01:58


In [31]:
import LinearAlgebra.norm
rel(x,y) = norm(x-y)/norm(x)
@show rel(Z_exact, Z_bp)
@show rel(p_ex, p_bp)
@show rel(r_bp, r_exact)
@show rel(c_bp, c_exact);

rel(Z_exact, Z_bp) = 0.0001645055512241575
rel(p_ex, p_bp) = 6.294719669209858e-5
rel(r_bp, r_exact) = 0.00010784830338836459
rel(c_bp, c_exact) = 0.00010312280021259295
